In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor,ExtraTreesRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.feature_selection import RFE
#import lightgbm as lgb

In [33]:
train_data = pd.read_csv('diamonds_train.csv')
test_data = pd.read_csv('diamonds_test.csv')


In [34]:
train_data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.21,Premium,J,VS2,62.4,58.0,4268,6.83,6.79,4.25
1,0.32,Very Good,H,VS2,63.0,57.0,505,4.35,4.38,2.75
2,0.71,Fair,G,VS1,65.5,55.0,2686,5.62,5.53,3.65
3,0.41,Good,D,SI1,63.8,56.0,738,4.68,4.72,3.00
4,1.02,Ideal,G,SI1,60.5,59.0,4882,6.55,6.51,3.95


In [35]:
cortes = {'Ideal': 5, 'Premium': 4, 'Very Good': 3, 'Good': 2, 'Fair': 1}
colores = {'D': 7, 'E': 6, 'F': 5, 'G': 4, 'H': 3, 'I': 2, 'J': 1}
claridad = {'IF': 8, 'VVS1': 7, 'VVS2': 6, 'VS1': 5, 'VS2': 4, 'SI1': 3, 'SI2': 2, 'I1': 1}

In [36]:
train_data['cut'] = train_data['cut'].apply(lambda x: cortes[x])
test_data['cut'] = test_data['cut'].apply(lambda x: cortes[x])
train_data['color'] = train_data['color'].apply(lambda x: colores[x])
test_data['color'] = test_data['color'].apply(lambda x: colores[x])
train_data['clarity'] = train_data['clarity'].apply(lambda x: claridad[x])
test_data['clarity'] = test_data['clarity'].apply(lambda x: claridad[x])

In [37]:
train_data['volume'] = train_data['x'] * train_data['y'] * train_data['z']
test_data['volume'] = test_data['x'] * test_data['y'] * test_data['z']

mean=train_data.volume.mean()
train_data=train_data.replace({'volume': {0: mean}}) 
test_data=test_data.replace({'volume': {0: mean}})

In [74]:
X = train_data[['carat', 'cut','color', 'clarity' , 'depth', 'y']]
Y = train_data['price']

In [75]:
auto_model=DecisionTreeRegressor()
selector = RFE(auto_model, n_features_to_select=4)
selector = selector.fit(X, Y)
selector.support_

array([ True, False,  True,  True, False,  True])

In [76]:
selector.ranking_

array([1, 3, 1, 1, 2, 1])

In [77]:
sc = StandardScaler()
#X = sc.fit_transform(X)

In [78]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=42)

In [79]:
train1X, valX, train1Y, valY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 1500, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3,4, 5,6,7,8,9, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4,5,6,7,8,9]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = DecisionTreeRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(valX, valY)
rf_random.best_params_

In [19]:
probar= {'n_estimators': 1600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 10,
 'bootstrap': True,
 'criterion':'mse',
 'n_jobs' : -1}

probar1={'n_estimators': 1555,
 'min_samples_split': 9,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 10,
 'bootstrap': True,
 'n_jobs' : -1       
        }

In [80]:
model_f = DecisionTreeRegressor(random_state=42, max_depth=10, min_samples_split= 9, min_samples_leaf= 2)

In [81]:
model_f.fit(trainX, trainY)

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=9, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=42, splitter='best')

In [82]:
predict_f = model_f.predict(testX)

In [83]:
model_f.score(testX, testY)

0.975280624060043

In [84]:
mean_squared_error(predict_f, testY)**0.5

634.4682716074809

In [25]:
test_df = test_data[['carat', 'color', 'clarity']]
test_df = sc.fit_transform(test_df)
final_predictions = model_f.predict(test_df)

In [26]:
submission = pd.DataFrame({
        "id": test_data["id"],
        "price": final_predictions
    })
submission.to_csv('submission.csv', index=False)